# **Animal image classification model by Javin**

---


Using [google-images-download](https://github.com/hardikvasa/google-images-download#troubleshooting-errors-issues) to download the dataset from google images
</br>
<font size=1> *Documentation:[Example documentation](https://google-images-download.readthedocs.io/en/latest/examples.html#) *</font>

In [ ]:
! pip install googleimagedownloader
! sudo apt-get install python3-setuptools
! pip install torchvision
! wget https://chromedriver.storage.googleapis.com/2.42/chromedriver_linux64.zip  && unzip chromedriver_linux64

In [18]:
import os
from google.colab import drive # this will be our driver
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [21]:
colab_path = '/gdrive/../content/'

In [22]:
chromedriver_path = '/gdrive/../content/chromedriver'

In [23]:
# Importing necessary packages

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np
import torch

import helper

import matplotlib.pyplot as plt

import time

In [24]:
from torchvision import datasets, transforms

# Defining a transform to normalize the data
transform = transforms.Compose([transforms.Resize(255),
                                transforms.CenterCrop(224),
                                transforms.ToTensor()])


Note: If google_images_download importing fails, check version of chrom driver as it should match with googleimagedownloader library

In [ ]:
from google_images_download import google_images_download  #importing the library

keyws = ['Dogs', 'Cats', 'Snakes', 'Turtle', 'Wasp']
limit   = 60
chromedriver = chromedriver_path
offset  = None  # how many links to skip
color_type	= None # color type you want to apply to the images.[full-color, black-and-white, transparent]
size    = medium #relative size of the image to be downloaded. [large, medium, icon, >400*300, >640*480, >800*600, >1024*768, >2MP, >4MP, >6MP, >8MP, >10MP, >12MP, >15MP, >20MP, >40MP, >70MP]
usage_rights	= 'labeled-for-reuse' 

Creating a config file

In [ ]:
arguments = [
        {
            "keywords" : keyws[0],
            "limit" :limit,
            "chromedriver":chromedriver,
            "offset" : offset,
            "color_type" : color_type,
            "size" : size,
            "usage_rights" : usage_rights
        },
        {
            "keywords" : keyws[1],
            "limit" :limit,
            "chromedriver":chromedriver,
            "offset" : offset,
            "color_type" : color_type,
            "size" : size,
            "usage_rights" : usage_rights
        },
        {
            "keywords" : keyws[2],
            "limit" :limit,
            "chromedriver":chromedriver,
            "offset" : offset,
            "color_type" : color_type,
            "size" : size,
            "usage_rights" : usage_rights
        },
        {
            "keywords" : keyws[3],
            "limit" :limit,
            "chromedriver":chromedriver,
            "offset" : offset,
            "color_type" : color_type,
            "size" : size,
            "usage_rights" : usage_rights
        },
        {
            "keywords" : keyws[4],
            "limit" :limit,
            "chromedriver":chromedriver,
            "offset" : offset,
            "color_type" : color_type,
            "size" : size,
            "usage_rights" : usage_rights
        }
    ]

In [ ]:
response  = google_images_download.googleimagesdownload()   #class instantiation
response.download(arguments.json)  

In [ ]:
for animal_name in keyws:
  dataset_path = 'downloads/' + animal_name + '/'
  dataset_dir = [ dataset_path + img_name for img_name in os.listdir( colab_path + dataset_path ) ]

dataset = datasets.ImageFolder(dataset_dir, transform=transform)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)# Run this to test your data loader
images, labels = next(iter(dataloader))
helper.imshow(images[0], normalize=False)

In [ ]:
data_dir = dataset_dir

# TODO: Define transforms for the training data and testing data
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor()]) 

test_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor()])


# Pass transforms in here, then run the next cell to see how the transforms look
train_data = datasets.ImageFolder(data_dir + '/train', transform=train_transforms)
test_data = datasets.ImageFolder(data_dir + '/test', transform=test_transforms)

trainloader = torch.utils.data.DataLoader(train_data, batch_size=32)
testloader = torch.utils.data.DataLoader(test_data, batch_size=32)

In [ ]:
model = models.densenet121(pretrained=True)
model

In [ ]:
# Freeze parameters so we don't backprop through them
for param in model.parameters():
    param.requires_grad = False

from collections import OrderedDict
classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(1024, 500)),
                          ('relu', nn.ReLU()),
                          ('fc2', nn.Linear(500, 2)),
                          ('output', nn.LogSoftmax(dim=1))
                          ]))
    
model.classifier = classifier

In [ ]:
for device in ['cpu', 'cuda']:

    criterion = nn.NLLLoss()
    # Only train the classifier parameters, feature parameters are frozen
    optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

    model.to(device)

    for ii, (inputs, labels) in enumerate(trainloader):

        # Move input and label tensors to the GPU
        inputs, labels = inputs.to(device), labels.to(device)

        start = time.time()

        outputs = model.forward(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        if ii==3:
            break
        
    print(f"Device = {device}; Time per batch: {(time.time() - start)/3:.3f} seconds")

In [ ]:
# Use GPU if it's available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.densenet121(pretrained=True)

# Freeze parameters so we don't backprop through them
for param in model.parameters():
    param.requires_grad = False
    
model.classifier = nn.Sequential(nn.Linear(1024, 256),
                                 nn.ReLU(),
                                 nn.Dropout(0.2),
                                 nn.Linear(256, 2),
                                 nn.LogSoftmax(dim=1))

criterion = nn.NLLLoss()

# Only train the classifier parameters, feature parameters are frozen
optimizer = optim.Adam(model.classifier.parameters(), lr=0.003)

model.to(device);

In [ ]:
epochs = 1
steps = 0
running_loss = 0
print_every = 5
for epoch in range(epochs):
    for inputs, labels in trainloader:
        steps += 1
        # Move input and label tensors to the default device
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        logps = model.forward(inputs)
        loss = criterion(logps, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        
        if steps % print_every == 0:
            test_loss = 0
            accuracy = 0
            model.eval()
            with torch.no_grad():
                for inputs, labels in testloader:
                    inputs, labels = inputs.to(device), labels.to(device)
                    logps = model.forward(inputs)
                    batch_loss = criterion(logps, labels)
                    
                    test_loss += batch_loss.item()
                    
                    # Calculate accuracy
                    ps = torch.exp(logps)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
                    
            print(f"Epoch {epoch+1}/{epochs}.. "
                  f"Train loss: {running_loss/print_every:.3f}.. "
                  f"Test loss: {test_loss/len(testloader):.3f}.. "
                  f"Test accuracy: {accuracy/len(testloader):.3f}")
            running_loss = 0
            model.train()